In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os

cwd = os.getcwd()
dataset_dir = "../input/maskdataset/MaskDataset"

# Training Directory
training_dir = os.path.join(dataset_dir, 'training')

# Validation Directory
valid_dir = os.path.join(dataset_dir, 'validation')

### Data Augmentation

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
import tensorflow as tf
import numpy as np

# Create generators to read images from dataset directory

SEED = 1234
tf.random.set_seed(SEED)  

# Batch size
bs = 8

# Img shape
img_h = 408
img_w = 612

num_classes = 3

# Training
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(img_h,img_w),
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors

# Validation
valid_gen = valid_data_gen.flow_from_directory(valid_dir,
                                               target_size=(img_h,img_w),
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

# Test
test_dir = os.path.join(dataset_dir, 'test')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             target_size=(img_h,img_w),
                                             batch_size=bs, 
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

Found 4774 images belonging to 3 classes.
Found 840 images belonging to 3 classes.
Found 0 images belonging to 0 classes.


### Create Dataset objects
#### GENERATOR:
* Shuffle            
* Normalize images   
* One-hot encoding    <- for categorical cross entropy
* Divide in batches  

In [5]:
# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
# Repeat
train_dataset = train_dataset.repeat()



# Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
# Repeat
valid_dataset = valid_dataset.repeat()



# Test
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
# Repeat
test_dataset = valid_dataset.repeat()

In [6]:
decide_class_indices = False

if decide_class_indices:
    classes = [ "NO PERSON in the image is wearing a mask",        #0
                "ALL THE PEOPLE in the image are wearing a mask",  #1
                "SOMEONE in the image is not wearing a mask"  ]    #2
else:
    classes = None
    
train_gen.class_indices
valid_gen.class_indices
test_gen.class_indices

{'0': 0, '1': 1, '2': 2}

{'0': 0, '1': 1, '2': 2}

{}

### Model

In [7]:
# Architecture: Features extraction -> Classifier

start_f = 10
depth = 7

model = tf.keras.Sequential()

# Features extraction
for index in range(depth):
    if index == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape = [None]

    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 2
    
# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [8]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 408, 612, 10)      280       
_________________________________________________________________
re_lu (ReLU)                 (None, 408, 612, 10)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 204, 306, 10)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 204, 306, 20)      1820      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 204, 306, 20)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 102, 153, 20)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 102, 153, 40)      7

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 10) dtype=float32, numpy=
 array([[[[ 1.72284245e-02, -6.13156706e-02,  3.69711667e-02,
           -1.16022877e-01, -3.15517485e-02,  1.83968410e-01,
            3.27035785e-03, -6.39429390e-02,  1.26293883e-01,
            3.86342555e-02],
          [ 1.21748433e-01, -7.58306682e-03,  3.95796746e-02,
           -2.16174245e-01, -2.91424394e-02,  2.16599867e-01,
            2.17881992e-01, -1.60112560e-01,  2.23997399e-01,
            1.27432123e-01],
          [-1.26756012e-01, -4.98522818e-03,  6.02933019e-02,
            2.88672596e-02, -4.36305404e-02, -4.58800346e-02,
            8.27440768e-02,  1.24156043e-01, -2.46743411e-02,
           -2.00707555e-01]],
 
         [[-1.15060918e-01, -8.81749392e-02, -8.27195048e-02,
           -1.39036402e-01,  1.47266880e-01,  4.18123752e-02,
           -5.68877012e-02,  1.55500576e-01, -1.17564380e-01,
            1.45892188e-01],
          [-1.52862310e-01, -1.05426461e-01, -3.94608527e-02,
 

### Optimization parameters


In [9]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics

metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [10]:
from datetime import datetime


cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

In [11]:
model.fit( x=train_dataset,
           epochs=100,
           steps_per_epoch=len(train_gen),
           validation_data=valid_dataset,
           validation_steps=len(valid_gen),
           callbacks=callbacks)

Epoch 1/100
597/597 [==============================] - 298s 499ms/step - loss: 1.1004 - accuracy: 0.3398 - val_loss: 1.0985 - val_accuracy: 0.3357
Epoch 2/100
597/597 [==============================] - 296s 496ms/step - loss: 1.0988 - accuracy: 0.3408 - val_loss: 1.0985 - val_accuracy: 0.3393
Epoch 3/100
597/597 [==============================] - 297s 497ms/step - loss: 1.0988 - accuracy: 0.3448 - val_loss: 1.0984 - val_accuracy: 0.3393
Epoch 4/100
597/597 [==============================] - 287s 481ms/step - loss: 1.0975 - accuracy: 0.3586 - val_loss: 1.1020 - val_accuracy: 0.3500
Epoch 5/100
597/597 [==============================] - 275s 460ms/step - loss: 1.0417 - accuracy: 0.4481 - val_loss: 0.9284 - val_accuracy: 0.5262
Epoch 6/100
597/597 [==============================] - 256s 429ms/step - loss: 0.8907 - accuracy: 0.5630 - val_loss: 0.7687 - val_accuracy: 0.6500
Epoch 7/100
597/597 [==============================] - 260s 436ms/step - loss: 0.7922 - accuracy: 0.6096 - val_loss: 0

In [14]:
import os
import pandas as pd


def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'
    with open(os.path.join(results_dir, csv_fname), 'w') as f:
        f.write('Id,Category\n')
        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


            
test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in os.listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe( images,
                                               test_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED )


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i += 1


create_csv(results)

Found 450 validated image filenames belonging to 1 classes.
57/57 [==============================] - 21s 369ms/step
